# Titanic Kaggle

In [51]:
import fancyimpute
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import warnings
import xgboost as xgb

warnings.filterwarnings("ignore")
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

The first step is going to be to combine the training and test sets so that any data transformations / feature engineering is easily applied to both. Only the training set is labeled, so I will create values of -999 for `Survived` in the test subset of data.

In [52]:
data = pd.concat(
    [train.assign(Train = 1), 
    test.assign(Train = 0).assign(Survived = -999)[list(train) + ['Train']]]
)

## Feature engineering

Dealing with `Name` - create last name, title, family features.

In [53]:
extract_lastname = lambda x: x.split(',')[0]

def extract_title(x):
    title = x.split(',')[1].split('.')[0][1:]
    if title in ['Mlle', 'Ms']:
        title = 'Miss'
    elif title == 'Mme':
        title = 'Mrs'
    elif title in ['Rev', 'Dr', 'Major', 'Col', 'Capt', 'Jonkheer', 'Dona']:
        title = 'Esteemed'
    elif title in ['Don', 'Lady', 'Sir', 'the Countess']:
        title = 'Royalty'
    return title
    
data = (data
    .assign(LastName = lambda x: x.Name.map(extract_lastname))
    .assign(Title = lambda x: x.Name.map(extract_title))
    .assign(FamSize = lambda x: x.SibSp + x.Parch + 1)
    .assign(Family = lambda x: [a + '_' + str(b) for a, b in 
                                zip(list(x.LastName), list(x.FamSize))])
    .drop(['Name', 'SibSp', 'Parch', 'LastName'], axis = 1)
)

Dealing with `Ticket` - to reduce overfitting, create dummy variables for tickets that are shared among two or more passengers.

In [54]:
def ticket_counts(data):
    ticket_to_count = dict(data.Ticket.value_counts())
    data['TicketCount'] = data['Ticket'].map(ticket_to_count.get)
    data['Ticket'] = np.where(data['TicketCount'] > 1, data['Ticket'], np.nan)
    return data.drop(['TicketCount'], axis = 1)

data = data.pipe(ticket_counts)

Dealing with the `Cabin` feature - creating a deck feature (the letter in the cabin name).

In [55]:
first_letter = np.vectorize(lambda x: x[:1]) 

data = (data
        .assign(Deck = lambda x: np.where(
            pd.notnull(x.Cabin), first_letter(x.Cabin.fillna('z')), x.Cabin))
        .assign(Deck = lambda x: np.where(x.Deck == 'T', np.nan, x.Deck))
        .drop(['Cabin'], axis = 1)
)

Drop columns we don't need, convert Sex to a binary variable.

In [56]:
data = (data
        .drop(['PassengerId'], axis = 1)
        .assign(Sex = lambda x: np.where(x.Sex == 'male', 1, 0))
)

Create dummy variables for categorical features.

In [57]:
def create_dummy_nans(data, col_name):
    deck_cols = [col for col in list(data) if col_name in col]
    for deck_col in deck_cols:
        data[deck_col] = np.where(
            data[col_name + 'nan'] == 1.0, np.nan, data[deck_col])
    return data.drop([col_name + 'nan'], axis = 1)

data = (data
        .assign(Pclass = lambda x: x.Pclass.astype(str))
        .pipe(pd.get_dummies, columns = ['Pclass', 'Family', 'Title', 'Ticket'])
        .pipe(pd.get_dummies, columns = ['Deck'], dummy_na = True)
        .pipe(pd.get_dummies, columns = ['Embarked'], dummy_na = True)
        .pipe(create_dummy_nans, 'Deck_')
        .pipe(create_dummy_nans, 'Embarked_')
)

Impute missing values.

In [58]:
def impute(data):
    impute_missing = data.drop(['Survived', 'Train'], axis = 1)
    impute_missing_cols = list(impute_missing)
    filled_soft = fancyimpute.MICE().complete(np.array(impute_missing))
    results = pd.DataFrame(filled_soft, columns = impute_missing_cols)
    results['Train'] = list(data['Train'])
    results['Survived'] = list(data['Survived'])
    assert results.isnull().sum().sum() == 0, 'Not all NAs removed'
    return results

data = data.pipe(impute)
print 'Number of NAs:', data.isnull().sum().sum()

[MICE] Completing matrix with shape (1309, 1167)
[MICE] Starting imputation round 1/110, elapsed time 0.032
[MICE] Starting imputation round 2/110, elapsed time 4.083
[MICE] Starting imputation round 3/110, elapsed time 7.139
[MICE] Starting imputation round 4/110, elapsed time 11.534
[MICE] Starting imputation round 5/110, elapsed time 14.098
[MICE] Starting imputation round 6/110, elapsed time 16.017
[MICE] Starting imputation round 7/110, elapsed time 17.391
[MICE] Starting imputation round 8/110, elapsed time 18.623
[MICE] Starting imputation round 9/110, elapsed time 20.098
[MICE] Starting imputation round 10/110, elapsed time 24.462
[MICE] Starting imputation round 11/110, elapsed time 26.723
[MICE] Starting imputation round 12/110, elapsed time 30.069
[MICE] Starting imputation round 13/110, elapsed time 33.560
[MICE] Starting imputation round 14/110, elapsed time 36.693
[MICE] Starting imputation round 15/110, elapsed time 38.240
[MICE] Starting imputation round 16/110, elapsed

Split into separate training and predicting sets.

In [59]:
outcomes = np.array(data.query('Train == 1')['Survived'])
train = (data.query('Train == 1')
         .drop(['Train', 'Survived'], axis = 1))
to_predict = (data.query('Train == 0')
              .drop(['Train', 'Survived'], axis = 1))

Further, randomly split the training set into training and test sets using hold-out cross validation.

In [60]:
X_train, X_test, y_train, y_test = train_test_split(
    train, outcomes, test_size = 0.2, random_state = 50)


def train_test_model(model, hyperparameters, X_train, X_test, y_train, y_test,
                    folds = 5):
    """
    Given a [model] and a set of possible [hyperparameters], along with 
    matricies corresponding to hold-out cross-validation, returns a model w/ 
    optimized hyperparameters, and prints out model evaluation metrics.
    """
    optimized_model = GridSearchCV(model, hyperparameters, cv = folds, n_jobs = -1)
    optimized_model.fit(X_train, y_train)
    predicted = optimized_model.predict(X_test)
    print 'Optimized parameters:', optimized_model.best_params_
    print 'Model accuracy (hold-out):', optimized_model.score(X_test, y_test)
    kfold_score = np.mean(cross_val_score(
            optimized_model.best_estimator_, np.append(X_train, X_test, axis = 0), 
            np.append(y_train, y_test), cv = folds))
    print 'Model accuracy ({0}-fold):'.format(str(folds)), kfold_score, '\n'
    return optimized_model


def create_submission(name, model, train, outcomes, to_predict):
    """
    Train [model] on [train] and predict the probabilties on [test], and
    format the submission according to Kaggle.
    """
    model.fit(np.array(train), outcomes)
    probs = model.predict(np.array(to_predict))
    results = pd.DataFrame(probs, columns = ['Survived'])
    results['PassengerId'] = list(pd.read_csv('data/test.csv')['PassengerId'])
    (results[['PassengerId', 'Survived']]
        .to_csv('submissions/' + name, index = False))
    return None

First model: random forest - scores ~ .799 on the public leaderboard, ~ 0.841 in local 5-fold CV.

In [61]:
%%time
rf_model = train_test_model(
    RandomForestClassifier(), {'n_estimators': [500, 800, 1000]}, 
    X_train, X_test, y_train, y_test)

Optimized parameters: {'n_estimators': 800}
Model accuracy (hold-out): 0.793296089385
Model accuracy (5-fold): 0.822658094388 

CPU times: user 16 s, sys: 188 ms, total: 16.2 s
Wall time: 26.4 s


Second model: logistic regression - scores ~ .799 on the public leaderboard, 0.845 in local 5-fold CV.

In [62]:
%%time
lr_model = train_test_model(
    LogisticRegression(), {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}, 
    X_train, X_test, y_train, y_test)

Optimized parameters: {'C': 100}
Model accuracy (hold-out): 0.810055865922
Model accuracy (5-fold): 0.831596358165 

CPU times: user 396 ms, sys: 60 ms, total: 456 ms
Wall time: 3.07 s


Third model: SVM w/ Gaussian kernal - scores ~ .770 on the public leaderboard, ~ .802 in local 5-fold CV.

In [65]:
%%time
svm_model = train_test_model(
    SVC(), {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}, 
    X_train, X_test, y_train, y_test)

Optimized parameters: {'C': 100}
Model accuracy (hold-out): 0.804469273743
Model accuracy (5-fold): 0.784505638192 

CPU times: user 3.2 s, sys: 84 ms, total: 3.29 s
Wall time: 7.1 s


Fourth model: Gradient Boosted Trees - scores ~.794 on the public leaderboard, .828 in local 5-fold CV.

In [22]:
xbg_param = {
    'learning_rate' : 0.025,
    'n_estimators' : 1000,
    'max_depth' : 5,
    'gamma' : 0,
    'subsample' : 0.8,
    'colsample_bytree' : 0.8,
    'objective' : 'binary:logistic',
    'nthread' : 4,
    'seed' : 27
}

xgb1 = xgb.XGBClassifier( **xbg_param )

xgtrain = xgb.DMatrix(np.array(train), label = np.array(outcomes))
cvresult = xgb.cv(xbg_param, xgtrain, num_boost_round = xgb1.get_params()['n_estimators'],
                 nfold = 5, metrics = 'error', early_stopping_rounds = 50, verbose_eval = True)

[0]	train-error:0.139045+0.00652761	test-error:0.192135+0.0223028
[1]	train-error:0.13455+0.00630604	test-error:0.18764+0.0153239
[2]	train-error:0.130337+0.00337075	test-error:0.177528+0.0135765
[3]	train-error:0.126124+0.00257405	test-error:0.168539+0.0158901
[4]	train-error:0.127528+0.00578402	test-error:0.168539+0.0123084
[5]	train-error:0.124719+0.0027232	test-error:0.170786+0.0126124
[6]	train-error:0.124719+0.00520978	test-error:0.173033+0.0108937
[7]	train-error:0.126404+0.00502484	test-error:0.174157+0.00794505
[8]	train-error:0.124719+0.00473379	test-error:0.173033+0.0114585
[9]	train-error:0.12191+0.00513351	test-error:0.170786+0.0149062
[10]	train-error:0.124158+0.0036191	test-error:0.170786+0.0153239
[11]	train-error:0.122753+0.00302507	test-error:0.170786+0.0144766
[12]	train-error:0.120506+0.00550445	test-error:0.161797+0.0195906
[13]	train-error:0.119663+0.00473372	test-error:0.166292+0.019655
[14]	train-error:0.119663+0.00286477	test-error:0.166292+0.019655
[15]	train-

In [24]:
%%time
gbm_model = train_test_model(
    xgb.XGBClassifier(learning_rate = 0.025, n_estimators = 99), 
    {'max_depth':range(3, 10, 2), 'min_child_weight':range(1, 6, 2),
    'gamma': [i / 10.0 for i in range(0, 5)], 'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]}, 
    np.array(X_train), np.array(X_test), y_train, y_test)

Optimized parameters: {'reg_alpha': 1e-05, 'max_depth': 3, 'gamma': 0.1, 'min_child_weight': 5}
Model accuracy (hold-out): 0.826815642458
Model accuracy (5-fold): 0.82828876791 

CPU times: user 34 s, sys: 648 ms, total: 34.7 s
Wall time: 36min 11s


In [67]:
%%time
gbm_model = xgb.XGBClassifier(learning_rate = 0.025, n_estimators = 500)
gbm_model.fit(X_train, y_train)
gbm_model.predict(X_test)
print gbm_model.score(X_test, y_test)

gbm_model = xgb.XGBClassifier(learning_rate = 0.025, n_estimators = 500).fit(train, outcomes)

0.815642458101
CPU times: user 6.43 s, sys: 0 ns, total: 6.43 s
Wall time: 5.35 s


In [75]:
gbm_model.predict_proba(np.array(X_test))

array([[ 0.03846174,  0.96153826],
       [ 0.69000459,  0.30999538],
       [ 0.91415739,  0.08584262],
       [ 0.43874055,  0.56125945],
       [ 0.91110843,  0.08889156],
       [ 0.54853308,  0.45146695],
       [ 0.98482352,  0.01517646],
       [ 0.95065665,  0.04934332],
       [ 0.03389198,  0.96610802],
       [ 0.28530294,  0.71469706],
       [ 0.07155484,  0.92844516],
       [ 0.09907222,  0.90092778],
       [ 0.96811438,  0.03188561],
       [ 0.24182338,  0.75817662],
       [ 0.94996959,  0.0500304 ],
       [ 0.83727479,  0.16272523],
       [ 0.02925879,  0.97074121],
       [ 0.08812642,  0.91187358],
       [ 0.57545519,  0.42454481],
       [ 0.22533637,  0.77466363],
       [ 0.71841681,  0.28158322],
       [ 0.32005388,  0.67994612],
       [ 0.07019538,  0.92980462],
       [ 0.06015736,  0.93984264],
       [ 0.27753407,  0.72246593],
       [ 0.94475341,  0.05524658],
       [ 0.92414546,  0.07585452],
       [ 0.11311734,  0.88688266],
       [ 0.04283178,

## Creating an ensemble

In [38]:
train.head()

,Sex,Age,Fare,FamSize,Pclass_1,Pclass_2,Pclass_3,Family_Abbott_3,Family_Allison_4,Family_Andersson_7,...,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Embarked_C,Embarked_Q,Embarked_S
0,1.0,22.0,7.2500,2.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,1.000000,0.0,0.0,0.0,1.0
1,0.0,38.0,71.2833,2.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,1.0,0.0,0.000000,0.000000,0.0,1.0,0.0,0.0
2,0.0,26.0,7.9250,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,1.000000,0.0,0.0,0.0,1.0
3,0.0,35.0,53.1000,2.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,1.0,0.0,0.000000,0.000000,0.0,0.0,0.0,1.0
4,1.0,35.0,8.0500,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.058891,0.0,0.0,0.100131,0.840978,0.0,0.0,0.0,1.0


## Export results

In [63]:
create_submission('lr2_model.csv', lr_model, train, outcomes, to_predict)

In [64]:
create_submission('rf2_model.csv', rf_model, train, outcomes, to_predict)

In [ ]:
create_submission('svm_model.csv', svm_model, train, outcomes, to_predict)

In [68]:
create_submission('gbm2_model.csv', gbm_model, train, outcomes, to_predict)

## Creating a majority voting ensemble from the exported models

In [70]:
lr = pd.read_csv('submissions/lr2_model.csv')
rf = pd.read_csv('submissions/rf_model.csv')
svm = pd.read_csv('submissions/svm_model.csv')
gbm = pd.read_csv('submissions/gbm_model.csv')

In [71]:
ensemble = rf.copy()[['PassengerId']]
ensemble['Survived'] = 0

def add_to_ensemble(data, ensemble):
    data = data.copy()
    data['Survived'] = np.where(data['Survived'] == 0, -1, 1)
    ensemble['Survived'] = ensemble['Survived'] + data['Survived']
    return ensemble

# The random forest model gets 2 votes
ensemble = add_to_ensemble(lr, ensemble)
ensemble = add_to_ensemble(lr, ensemble)
ensemble = add_to_ensemble(lr, ensemble)
ensemble = add_to_ensemble(rf, ensemble)
ensemble = add_to_ensemble(gbm, ensemble)

In [72]:
ensemble['Survived'] = np.where(ensemble['Survived'] > 0, 1, 0)

In [73]:
ensemble.to_csv('submissions/ensemble_majority2.csv', index = False)